# Poner en orden los datos y hacer un EDA rapido

Importamos las librerias

In [29]:
import os
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from ipywidgets import interact
import geopandas as gpd

In [30]:
df_f = pd.read_csv('ENIGH.csv')
mapa = os.path.join('..', '..', 'ingCaracteristicasProyecto_', 'data', 'raw', 'MAPA', '2023_1_00_ENT.shp')

'''Cargar el CSV y el archivo .shp'''
datos_mapa = gpd.read_file(mapa, encoding='latin-1')

df_f['Entidad'] = df_f['entidad'].astype(int)
datos_mapa['CVE_ENT'] = datos_mapa['CVE_ENT'].astype(int)

    # Fusionar los DataFrames en la columna 'Entidad' y 'CVE_ENT'
df_f = pd.merge(df_f, datos_mapa, left_on='Entidad', right_on='CVE_ENT', how='left')

    # Convertir a GeoDataFrame y renombrar columnas
df_f = gpd.GeoDataFrame(df_f, geometry='geometry')
df_f = df_f.drop(columns=['CVE_ENT', 'CVEGEO', 'geometry', 'entidad'])
df_f= df_f.rename(columns={'NOMGEO': 'Entidad', 'Entidad': 'C_Entidad', 'folioviv':'Folio', 'edad':'Edad',	'sexo':'Sexo', 'year':'Fecha', 'alfabetism':'Alfabetismo',	'generacion':'Generacion'})
df_f = df_f[['Folio', 'Edad', 'Sexo', 'C_Entidad', 'Entidad', 'Fecha', 'Alfabetismo', 'Generacion']]
# Cambiar los tipos de datos
df_f['Folio'] = df_f['Folio'].astype('object')
df_f['Edad'] = df_f['Edad'].astype('int64')    # Asegurar que es entero
df_f['Sexo'] = df_f['Sexo'].astype('category')  # Convertir a categoría
df_f['C_Entidad'] = df_f['C_Entidad'].astype('category')  # Convertir a categoría
df_f['Alfabetismo'] = df_f['Alfabetismo'].astype('category')
df_f['Fecha'] = pd.to_datetime(df_f['Fecha'], format='%Y')


df_f.to_parquet('enigh_processed.parquet')

A continuacion `df_1` son los datos de salud mental, alcoholismo y tabaquismo, mientras que `df_2` son los resultados de la encuesta del **enigh**.

In [31]:
df_1 = pd.read_parquet('Ensanut-data-processed.parquet')
df_2 = pd.read_parquet('enigh_processed.parquet')

df_1['Sexo'] = df_1['Sexo'].astype('category')
df_1['C_Entidad'] = df_1['C_Entidad'].astype('category')
df_1['Fecha'] = pd.to_datetime(df_1['Fecha'], errors='coerce')
df_1['Año'] = df_1['Fecha'].dt.year
df_1 = df_1.dropna(subset=['Edad']).reset_index(drop=True)
df_1['Edad'] = df_1['Edad'].astype('int')


df_2['Sexo'] = df_2['Sexo'].astype('category')
df_2['C_Entidad'] = df_2['C_Entidad'].astype('category')
df_2['Fecha'] = pd.to_datetime(df_2['Fecha'], format='%Y')
df_2['Año'] = df_2['Fecha'].dt.year
df_2['Generacion'] = df_2['Generacion'].astype('category')

In [32]:
df_1.head()

,Folio,Edad,Sexo,C_Entidad,Entidad,Fecha,Atentar_contras_si,Depresion,Tristeza,Cuantos cigarrillos (numero),Frecuencia emborrachar,Edad_ii,Atentar_contras_si_ii,Depresion_ii,Tristeza_ii,Cuantos cigarrillos (numero)_ii,Frecuencia emborrachar_ii,Fecha_ii,Año
0,0100100210377005002,10,1,1,Aguascalientes,2000-09-09,NaN,NaN,NaN,<NA>,5,10,2,1,1,2,5,2000-09-09,2000.0
1,0100100210377005004,19,2,1,Aguascalientes,2000-09-09,NaN,NaN,NaN,<NA>,4,19,2,1,1,4,4,2000-09-09,2000.0
2,0100100210377016008,11,1,1,Aguascalientes,2000-09-09,NaN,NaN,NaN,<NA>,5,11,2,1,1,2,5,2000-09-09,2000.0
3,0100100210377016010,17,2,1,Aguascalientes,2000-09-09,NaN,NaN,NaN,<NA>,6,17,2,1,1,1,6,2000-09-09,2000.0
4,0100100210377016012,14,2,1,Aguascalientes,2000-09-09,NaN,NaN,NaN,<NA>,5,14,2,1,1,2,5,2000-09-09,2000.0


In [34]:
df_2['Año'].unique()

array([2016, 2018, 2020, 2022])

In [35]:
df_2['Generacion'].unique()

['Silent', 'Baby Boomer', 'Milenial', 'X', 'Z', 'Alpha', 'Greatest']
Categories (7, object): ['Alpha', 'Baby Boomer', 'Greatest', 'Milenial', 'Silent', 'X', 'Z']

In [36]:
def asignar_generacion(edad):
    if edad < 10:
        return "Alpha"
    elif 10 <= edad < 20:
        return "Z"
    elif 20 <= edad < 36:
        return "Milenial"
    elif 36 <= edad < 52:
        return "X"
    elif 52 <= edad < 71:
        return "Baby Boomer"
    elif 71 <= edad < 92:
        return "Silent"
    elif edad >= 92:
        return "Greatest"
    else:
        return "Unknown"

df_1['Generacion'] = df_1['Edad'].apply(asignar_generacion)
df_1.head()

,Folio,Edad,Sexo,C_Entidad,Entidad,Fecha,Atentar_contras_si,Depresion,Tristeza,Cuantos cigarrillos (numero),Frecuencia emborrachar,Edad_ii,Atentar_contras_si_ii,Depresion_ii,Tristeza_ii,Cuantos cigarrillos (numero)_ii,Frecuencia emborrachar_ii,Fecha_ii,Año,Generacion
0,0100100210377005002,10,1,1,Aguascalientes,2000-09-09,NaN,NaN,NaN,<NA>,5,10,2,1,1,2,5,2000-09-09,2000.0,Z
1,0100100210377005004,19,2,1,Aguascalientes,2000-09-09,NaN,NaN,NaN,<NA>,4,19,2,1,1,4,4,2000-09-09,2000.0,Z
2,0100100210377016008,11,1,1,Aguascalientes,2000-09-09,NaN,NaN,NaN,<NA>,5,11,2,1,1,2,5,2000-09-09,2000.0,Z
3,0100100210377016010,17,2,1,Aguascalientes,2000-09-09,NaN,NaN,NaN,<NA>,6,17,2,1,1,1,6,2000-09-09,2000.0,Z
4,0100100210377016012,14,2,1,Aguascalientes,2000-09-09,NaN,NaN,NaN,<NA>,5,14,2,1,1,2,5,2000-09-09,2000.0,Z


Vamos a dejar solo en los que el anio sea 2016, 2020 o 2022

In [37]:
df_1 = df_1[df_1['Año'].isin([2018, 2021, 2022])].reset_index(drop=True)
df_2 = df_2[df_2['Año'].isin([2018, 2020, 2022])].reset_index(drop=True)
df_1.head()

,Folio,Edad,Sexo,C_Entidad,Entidad,Fecha,Atentar_contras_si,Depresion,Tristeza,Cuantos cigarrillos (numero),Frecuencia emborrachar,Edad_ii,Atentar_contras_si_ii,Depresion_ii,Tristeza_ii,Cuantos cigarrillos (numero)_ii,Frecuencia emborrachar_ii,Fecha_ii,Año,Generacion
0,846,11,2,1,Aguascalientes,2018-07-31,2,1,1,<NA>,NaN,11,2,1,1,3,4,2018-07-31,2018.0,Z
1,211,12,2,1,Aguascalientes,2018-07-31,2,1,1,<NA>,NaN,12,2,1,1,3,4,2018-07-31,2018.0,Z
2,211,19,1,1,Aguascalientes,2018-07-31,2,1,1,<NA>,3,19,2,1,1,5,3,2018-07-31,2018.0,Z
3,211,15,1,1,Aguascalientes,2018-07-31,2,1,1,<NA>,NaN,15,2,1,1,3,4,2018-07-31,2018.0,Z
4,211,11,1,1,Aguascalientes,2018-07-31,2,1,1,<NA>,NaN,11,2,1,1,3,4,2018-07-31,2018.0,Z


In [38]:
df_1['Año'] = df_1['Año'].replace({2021: 2020})

df_1.loc[df_1["Año"] == 2020, "Fecha"] = "2020-07-31"

df_1['Fecha'] = pd.to_datetime(df_1['Fecha'], errors='coerce')

print(df_1['Año'].unique())


[2018. 2020. 2022.]


In [39]:
df_1.to_parquet('ensanut_processed.parquet')
df_2.to_parquet('enigh_processed.parquet')